In [19]:
import numpy as np

In [22]:
def my_feature_sign(S, y, bound):
    count1 = 1
    count2 = 0
    tmp = np.dot(S.T, S)
    theta = np.zeros(tmp.shape[0], dtype=np.int8)
    x = np.zeros(tmp.shape[0])
    STy = np.dot(S.T, y)
    grad = -2 * STy
    now_active = set()
    y2 = np.dot(y.T, y)
    
    done = 0
    while not done:
        if count2<1:
            tmpx = np.argmax(np.abs(grad)*(theta==0))
            if grad[tmpx] > bound:
                theta[tmpx] = -1
                now_active.add(tmpx)
            elif grad[tmpx] < -bound:
                theta[tmpx] = 1
                now_active.add(tmpx)
            
        index = np.array(sorted(now_active))
        tmp_H = tmp[np.ix_(index, index)]
        tmp_Sy = STy[index]
        thetaHat = theta[index]
        b = tmp_Sy-(bound*thetaHat)/2
        x_H = np.linalg.solve(np.atleast_2d(tmp_H), b)
        thetaNew = np.sign(x_H)
        oldx = x[index]
        changed = np.where(abs(thetaNew - thetaHat) > 1)[0]
        if len(changed) > 0:
            xb = x_H
            Sx2 = np.dot(x_H, np.dot(tmp_Sy,x_H))
            ab = np.dot(x_H, tmp_Sy)
            obj = (y2 + Sx2- 2 * ab + bound*abs(x_H).sum())
            for elem in changed:
                x = x_H[elem]
                y = oldx[elem]
                prop = y / (y - x)
                now = oldx - prop * (oldx - x_H)
                tmp1 = np.dot(now, np.dot(tmp_H, now))
                tmp2 = np.dot(now, tmp_Sy)
                costnow = y2 + (tmp1-2*tmp2+ bound*abs(now).sum())
                if costnow < obj:
                    obj = costnow
                    best_prop = prop
                    xb = now
        else:
            xb = x_H
        x[index] = xb
        index_zeros = index[np.abs(x[index]) < 1e-18]
        x[index_zeros] = 0.
        theta[index] = np.int8(np.sign(x[index]))
        now_active.difference_update(index_zeros)
        grad = - 2 * STy + 2 * np.dot(tmp, x)
        tmp_res = np.max(abs(grad[theta == 0]))
        #tmp_res=np.inf
        ntmp_res = np.max(abs(grad[theta != 0] + bound * theta[theta != 0]))
        
        if tmp_res > bound:
            count1=1
        else:
            count1=0
            
        if not np.allclose(ntmp_res,0):
            count2=1
        else:
            count2=0
        if count1+count2:
            done=0
        else:
            done = 1
    return x

In [24]:
S = np.array([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,1,0],[0,0,0,1,0,1]])

bound= 0.5
y = np.array([4,6,2,3])

x = my_feature_sign(S, y, bound)
x



array([3.75, 5.75, 1.75, 2.75, 0.  , 0.  ])

In [30]:
S = np.array([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,1,0],[0,0,0,1,0,1]])

bound= 0.3
y = np.array([4,6,2,3])

x = my_feature_sign(S, y, bound)
x

array([3.85, 5.85, 1.85, 2.85, 0.  , 0.  ])